In [5]:
import json
from pprint import pprint
import tinydb as tdb

In [1]:
dbname = 'data/sqliteDB.sqlite'

In [2]:
!rm {dbname}

rm: impossible de supprimer 'data/sqliteDB.sqlite': Aucun fichier ou dossier de ce type



## OSM: Requete OverPass turbo:

```sql
[out:json][timeout:500];
{{geocodeArea:France}}->.boundaryarea;

(
 node(area.boundaryarea)["ref:mhs"];
 way(area.boundaryarea)["ref:mhs"];
 relation(area.boundaryarea)["ref:mhs"];
);

out tags;
```
-- 20Mo

Rq: j'ai essayé avec `out meta;`, le fichier fait ~200Mo, et avec `out body;` ~90Mo... c'est surement mieux de séparer par département...


In [20]:
# Charge le fichier JSON:
filename = 'data/osmTags_France.json' 

with open(filename) as f:
    dataOsm = json.load(f)
    
dataOsm = dataOsm['elements']
len(dataOsm)

33071

In [21]:
# Garde seulement les monuments avec le code commençant par PA...
dataOsm = [ el for el in dataOsm if el['tags']['ref:mhs'].startswith('PA') ]
len(dataOsm)

32246

In [22]:
def flatten( dico ):
    flatdico = dico['tags']
    flatdico['idOSM'] = dico['id']
    flatdico['typeOSM'] = dico['type']
    
    return flatdico

In [23]:
# Garde que les tags - and flatten the dict 
dataOsm = [ flatten( el ) for el in dataOsm ]

In [46]:
dataOsm[31111]

{'building': 'yes',
 'building:levels': '4',
 'heritage': '3',
 'heritage:operator': 'mhs',
 'idOSM': 554059,
 'ref:mhs': 'PA00086328',
 'type': 'multipolygon',
 'typeOSM': 'relation',
 'wikidata': 'Q22921439'}

In [33]:
keys = set( k  for d in dataOsm for k in d.keys() )
print( keys )

{'payment:mastercard', 'festival:Avignon_off:2015', 'concentration_camp', 'brand:wikidata', 'source:maxspeed', 'factory:wood', 'population', 'toilets', 'geological', 'local_name', 'name:ba', 'mhs:inscription_date', 'alt_name:zh_hans', 'name:ku', 'name:viaduct', 'memorial:type', 'ms:inscription_dateate', 'lanes:backward', 'boat', 'smoothness', 'seamark:light:3:sequence', 'military', 'seamark:light:5:sector_end', 'building:part', 'int_name', 'addr:hamlet', 'alt_name', 'source:image', 'seamark:light:3:character', 'highway', 'seamark:light:2:range', 'railway:historic', 'seamark:light:1:colour', 'ref:RFF:ligne', 'railway:radio', 'monument', 'lcn_ref', 'name:el', 'bridge', 'name:wikipedia', 'b:type', 'denomination', 'maxheight', 'place', 'tracktype', 'ref:FR:LILLE', 'ref:sandre', 'cemetery:capacity', 'seamark:light:6:sector_end', 'CRNPDC:adresse2', 'name:tr', 'website:concerts', 'diocese', 'historic:building', 'vhf_channel', 'source:geometry', 'seamark:light:9:height', 'healthcare', 'typeOSM

In [26]:
# Push to tinyDB
db = tdb.TinyDB(dbname)
db.purge_table('osm')
osm = db.table('osm')
ids = osm.insert_multiple(dataOsm)
len(ids)

32246

## Requete Wikidata
```sql
SELECT DISTINCT ?item ?itemDescription ?merimee ?article  WHERE {
  ?item wdt:P380 ?merimee .
  
  OPTIONAL { ?article schema:about ?item .
  ?article schema:isPartOf <https://fr.wikipedia.org/> . }
   
  FILTER(STRSTARTS(?merimee, "PA")).
  
  SERVICE wikibase:label {
     bd:serviceParam wikibase:language "fr" .
 }
}
```
-- 7,6 Mo

In [27]:
# Charge le fichier JSON:
filename = 'data/wikidata_France.json' 

with open(filename) as f:
    dataWiki = json.load(f)
    
len(dataWiki)

44912

In [28]:
dataWiki[0]

{'article': 'https://fr.wikipedia.org/wiki/%C3%89glise_Saint-%C3%89tienne-du-Mont_de_Paris',
 'item': 'http://www.wikidata.org/entity/Q583251',
 'itemDescription': 'édifice religieux de Paris, en France',
 'merimee': 'PA00088414'}

In [29]:
# Push to tinyDB
db = tdb.TinyDB(dbname)
db.purge_table('wikidata')
wikidata = db.table('wikidata')
ids = wikidata.insert_multiple(dataWiki)
len(ids)

44912

# Merimee

In [35]:
# -- Charge le fichier JSON: --
# src: https://www.data.gouv.fr/fr/datasets/immeubles-proteges-au-titre-des-monuments-historiques/

#filename = 'data/liste-des-immeubles-proteges-au-titre-des-monuments-historiques.json' 
filename = 'data/merimee-MH.json' 

with open(filename) as f:
    dataMerimee = json.load(f)
    
print( len(dataMerimee) )

# convert keys to lower-case:
dataMerimee = [ { k.lower():v for k, v in d.items() }  for d in  dataMerimee ]

45268


In [36]:
dataMerimee[1]

{'adrs': '',
 'affe': '',
 'autr': '',
 'com': 'Aix-en-Othe',
 'dpro': '1980/11/14 : inscrit MH',
 'dpt': '10',
 'etud': 'Recensement immeubles MH',
 'insee': '10003',
 'ppro': 'Eglise de la Nativité (cad. AB 179) : inscription par arrêté du 14 novembre 1980',
 'ref': 'PA00078015',
 'reg': 'Champagne-Ardenne',
 'scle': '16e siècle ; 17e siècle ; 18e siècle',
 'stat': 'Propriété de la commune',
 'tico': 'Eglise de la Nativité'}

In [37]:
# Push to tinyDB

db = tdb.TinyDB(dbname)
db.purge_table('merimee')
merimee = db.table('merimee')

ids = merimee.insert_multiple(dataMerimee)

len(ids)

45268

In [38]:
keys = set( k  for d in merimee for k in d.keys() )
print( keys )

{'com', 'stat', 'affe', 'etud', 'autr', 'insee', 'dpro', 'scle', 'dpt', 'ref', 'adrs', 'tico', 'ppro', 'reg'}
